In [5]:
%pip install -U -q "google-generativeai>=0.8.3"
# chromadb

Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install langchain
!pip install langchain_experimental

In [13]:
import google.generativeai as genai
from IPython.display import Markdown
from langchain.utilities import SQLDatabase
import pandas as pd
import sqlite3
import os

In [14]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [15]:
# for m in genai.list_models():
#     if "embedContent" in m.supported_generation_methods:
#         print(m.name)

In [16]:
df = pd.read_csv('/kaggle/input/data-cb/data_cb.csv')
df.head()

,gcn_number,CIFNO,ACCTNO,fullname,MEGA_FIRST_DESC,cust_sex,cust_type,CFINSC,Birthplace,tipe_id,...,Nomor HP,M-Smile Flag,CC Flag,WM Flag,CD Flag,i-Bank Flag,flag_pep,ChannelPembukaan,ACOBUSER,flag_customer
0,ANI4588,A000026,144690803,T. Cahyono Andriani,INA,Laki-Laki,Individu,NaN,Semarang,KT,...,5.670861e+12,1,0,0,0,0,0,NaN,NaN,EXISTING
1,IWI21568,A000027,95899229,Jamalia Pertiwi,INA,Perempuan,Non-Individu,X,Pekanbaru,KT,...,NaN,0,0,0,0,0,0,NaN,NaN,EXISTING
2,ARI11575,A000029,963647598,Makara Lestari,INA,Perempuan,Individu,NaN,Palu,KT,...,6.116257e+12,0,1,0,0,0,0,NaN,NaN,EXISTING
3,SUR102745,A000035,726502253,Dt. Wasis Mansur,INA,Laki-Laki,Individu,S,Manado,SM,...,NaN,0,1,0,0,0,0,NaN,NaN,NTB
4,TMI4654,A000037,730039688,Umay Suryatmi,INA,Perempuan,Individu,NaN,Sabang,LL,...,NaN,1,0,0,0,0,0,NaN,NaN,NTB


In [18]:
# Connect to a new SQLite database
con = sqlite3.connect('my_data.db')

In [19]:
# Write the DataFrame to the SQL database
df.to_sql('cb', con, if_exists='replace', index=False)

1000

In [20]:
df_from_sql = pd.read_sql_query('SELECT * FROM cb limit 10', con)
# Print the data from the SQL database
print(df_from_sql)

  gcn_number    CIFNO     ACCTNO                 fullname MEGA_FIRST_DESC  \
0    ANI4588  A000026  144690803      T. Cahyono Andriani             INA   
1   IWI21568  A000027   95899229          Jamalia Pertiwi             INA   
2   ARI11575  A000029  963647598           Makara Lestari             INA   
3  SUR102745  A000035  726502253         Dt. Wasis Mansur             INA   
4    TMI4654  A000037  730039688            Umay Suryatmi             INA   
5  AIN101878  A000039  741393264          Gara Zulkarnain             INA   
6   RNO92041  A000041  149911317       H. Kambali Winarno             INA   
7   STU71277  A000042  547578352             Eko Pangestu             INA   
8   PSI72550  A000043  865338806  Himawan Mahendra, S.Psi             INA   
9  TRA121187  A000044  501011109            Ganda Saputra             INA   

    cust_sex     cust_type CFINSC   Birthplace tipe_id  ...      Nomor HP  \
0  Laki-Laki      Individu   None     Semarang      KT  ...  5.670861e+12  

In [21]:
db=SQLDatabase.from_uri('sqlite:////kaggle/working/my_data.db')
print(db)

In [23]:
# model = genai.GenerativeModel("gemini-1.5-flash-latest")
# answer = model.generate_content(prompt)

In [25]:
# # Fungsi untuk menghasilkan SQL menggunakan Gemini API
# def generate_sql(nl_query):
#     model = genai.GenerativeModel("gemini-1.5-flash-latest")
#     prompt = custom_prompt.format(query=nl_query)
#     prompt = """
#         You are interacting with a database containing a table:
#         - Columns: gcn_number, CIFNO, ACCTNO, fullname, cust_sex, cust_type, CFINSC, Birthplace, tipe_id

#         Translate the following natural language query into SQL:
#         {query}
#     """
#     response = model.generate_text(prompt=prompt, temperature=0.1)  # Memanggil API Gemini
#     return response.result  # Mendapatkan teks yang dihasilkan

In [41]:
# import genai

def generate_sql(nl_query):
  """Generates SQL query from a natural language query using the Gemini API.

  Args:
    nl_query: The natural language query to be translated.

  Returns:
    The generated SQL query.
  """

  model = genai.GenerativeModel("gemini-1.5-flash-latest")

  # Construct the prompt with clear instructions and table schema
  prompt = f"""
  You are interacting with a database containing a table with the following columns:
  - gcn_number
  - CIFNO
  - ACCTNO
  - fullname
  - cust_sex
  - cust_type
  - CFINSC
  - Birthplace
  - tipe_id

  Translate the following natural language query into SQL:
  {nl_query}
  """

  response = model.generate_content(prompt)
  return response.text

In [30]:
# custom_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
#         You are interacting with a database containing a table:
#         - Columns: gcn_number, CIFNO, ACCTNO, fullname, cust_sex, cust_type, CFINSC, Birthplace, tipe_id

#         Translate the following natural language query into SQL:
#         {query}
#     """
# )

In [40]:
# help(genai.GenerativeModel.generate_content)

In [39]:
# pip install --upgrade genai

In [42]:
# Fungsi generate_sql (asumsikan sudah didefinisikan seperti di atas)

# Query bahasa alami
nl_query = "berapa jumlah baris di tabel cb?"

# Menghasilkan SQL
generated_sql = generate_sql(nl_query)
print("Generated SQL:", generated_sql)

# Membuat koneksi ke database
conn = sqlite3.connect("my_data.db")  # Ganti dengan database Anda
cursor = conn.cursor()

# Eksekusi query untuk mendapatkan jumlah baris
try:
    cursor.execute("SELECT COUNT(*) FROM cb")
    num_rows = cursor.fetchone()[0]
    print("Jumlah baris pada tabel cb:", num_rows)

    # Untuk mendapatkan jumlah kolom, kita bisa menggunakan metadata database
    cursor.execute("PRAGMA table_info(cb)")
    num_cols = len(cursor.fetchall())
    print("Jumlah kolom pada tabel cb:", num_cols)

except Exception as e:
    print("Terjadi kesalahan:", e)

# Menutup koneksi database
conn.close()

Generated SQL: ```sql
SELECT COUNT(*) FROM cb;
```

Jumlah baris pada tabel cb: 1000
Jumlah kolom pada tabel cb: 53


In [43]:
# # Membuat LLM dari Gemini API
# llm = genai.LanguageModel(
#     model_name="gemini-1.5-flash-latest",  # Sesuaikan nama model yang digunakan
#     temperature=0.1,        # Parameter untuk kontrol kreativitas
# )

# # Prompt untuk menerjemahkan query natural language ke SQL
# custom_prompt = PromptTemplate(
#     input_variables=["query"],
#     template="""
#         You are interacting with a database containing a table:
#         - Columns: gcn_number, CIFNO, ACCTNO, fullname, cust_sex, cust_type, CFINSC, Birthplace, tipe_id

#         Translate the following natural language query into SQL:
#         {query}
#     """
# )

In [44]:
# # Membuat chain untuk menghasilkan SQL dari query natural language
# sql_chain = LLMChain(llm=llm, prompt=custom_prompt)

# # Contoh query dalam bahasa natural
# nl_query = "berapa jumlah baris dan kolom di table cb?"

# # Menghasilkan SQL dari query
# generated_sql = sql_chain.run(query=nl_query)
# print("Generated SQL:", generated_sql)

# # Membuat koneksi ke database (contoh: SQLite)
# conn = sqlite3.connect("example.db")  # Ganti dengan database Anda
# cursor = conn.cursor()

# # Eksekusi query yang dihasilkan
# try:
#     cursor.execute(generated_sql)
#     results = cursor.fetchall()
#     print("Query Results:")
#     for row in results:
#         print(row)
# except Exception as e:
#     print("Error executing SQL:", e)

# # Menutup koneksi database
# conn.close()